## This script sets up the model and performs all relevant model runs.

The time resolution is set using the nhours argument in the mkwargs dictionary. Use 1 for full detail.

In [11]:
import sys, os

import numpy as np
import pandas as pd

import grimsel.core.model_loop as model_loop
from grimsel.core.model_base import ModelBase as MB
import grimsel.auxiliary.aux_sql_func as aql
import grimsel.config as config
import grimsel.model_loop_modifier as model_loop_modifier
import grimsel.analysis.sql_analysis as sql_analysis

db = config.DATABASE

sc_out = 'out_replace_all_test'

In [2]:
connect_dict = dict(db=db,
                    password=config.PSQL_PASSWORD,
                    user=config.PSQL_USER,
                    port=config.PSQL_PORT,
                    host=config.PSQL_HOST)
sqlc = aql.sql_connector(**connect_dict)

# additional kwargs for the model
mkwargs = {
           'slct_encar': ['EL'],
           'slct_node': ['AT0', 'IT0', 'DE0', 'CH0', 'FR0'],
           'nhours': 168,
           'verbose_solver': False,
           'constraint_groups': MB.get_constraint_groups(excl=['chp', 'ror'])
           }

# additional kwargs for the i/o
iokwargs = {'resume_loop': False,
            'autocomplete_curtailment': True,
           }

nvr, nst = 30, 31
nsteps_default = [
                  ('swvr', nvr, np.arange),     # select vre share
                  ('swst', nst, np.arange),     # select storage capacity
                  ('swtc', 3, np.arange),       # select storage tech
                  ('swpt', 3, np.arange),       # select vre type
                  ('swyr', 5, np.arange),       # select meteo year
                  ('swco', 3, np.arange),       # select co2 emission price
                  ('swtr', 2, np.arange),       # cross-border transmission on/off
                  ('swrc', 26, np.arange)]      # select ramping cost

mlkwargs = {
            'sc_out': sc_out,
            'db': db,
            'nsteps': nsteps_default,
            'sql_connector': sqlc,
            }

sc_out = mlkwargs['sc_out']

ml = model_loop.ModelLoop(**mlkwargs, mkwargs=mkwargs, iokwargs=iokwargs)


swst_max = nst
dict_st = {nst: st for nst, st in enumerate(list(np.linspace(0, 0.3, swst_max)))}
swvr_max = nvr - 1
dict_vre = {nvr: vr for nvr , vr
            in enumerate(['default'] + list(np.linspace(0, 0.7, swvr_max)))}
dict_rc = {**{ii: ii/10 for ii in range(26)}, **{0: 1, 10: 0}}



self.slct_encar=['EL']
self.slct_pp_type=[]
self.slct_node=['AT0', 'IT0', 'DE0', 'CH0', 'FR0']
self.nhours=168
self.constraint_groups=['capacity', 'charging_level', 'chp_new', 'energy_aggregation', 'energy_constraint', 'hydro', 'monthly_total', 'objective', 'ramp_rate', 'supply', 'transmission', 'variables', 'yearly_cost']
Output schema:  out_replace_all_test ; resume loop= False

~~~~~~~~~~~~~~~   WARNING:  ~~~~~~~~~~~~~~~~
You are about to delete existing schema out_replace_all_test.
The maximum run_id is None.

Hit enter to proceed.

Dropping output schema:  out_replace_all_test
Reading input table def_month 
Reading input table def_week 
Reading input table parameter_month 
Reading input table def_plant 
Reading input table def_node filtered by nd in ['AT0', 'IT0', 'DE0', 'CH0', 'FR0']
Reading input table def_pp_type 
Reading input table def_encar filtered by ca in ['EL']
Reading input table def_plant filtered by nd_id in [0, 1, 2, 3, 4], pt_id in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

Assigning parameter supprof... ok.
Assigning parameter inflowprof... ok.
Assigning parameter priceprof... failed: set(s) ['fl_prof'] is/are empty or None.
Hydro parameters
Assigning parameter min_erg_mt_out_share... ok.
Assigning parameter max_erg_mt_in_share... ok.
Assigning parameter min_erg_share... ok.
Defining general parameters
Assigning parameter weight... ok.
Assigning parameter month_weight... ok.
Assigning parameter dmnd_sum... ok.
Assigning parameter grid_losses... ok.
Assigning parameter vc_dmnd_flex... ok.
Assigning parameter chp_cap_pwr_leg... ok.
Assigning parameter cap_trm_leg... ok.
Defining ramping parameters
Assigning parameter vc_ramp... ok.
Defining pp parameters
Assigning parameter ca_share_min... failed (column doesn't exist).
Assigning parameter ca_share_max... failed (column doesn't exist).
Assigning parameter pp_eff... ok.
Assigning parameter cf_max... ok.
Defining fuel parameters
Assigning parameter erg_inp... ok.
Assigning parameter erg_chp... ok.
Assigning 

## Reduce output
Don't write hourly data.

In [4]:
ml.io.var_sy = []#[par for par in ml.io.var_sy if 'pwr' in par[0] or 'pwr_st_ch' in par[0]]
ml.io.par = [par for par in ml.io.par if not 'prof' in par[0]]
ml.io.var_tr = [var for var in ml.io.var_tr if 'erg' in var[0]]
ml.io.dual = []
print(ml.io.var_sy)

[]


## Some filtering of the ml.df_def_loop table

The ModelLoop object created a table with all combinations of the nsteps_default as model runs. 

Here this table is filtered so we don't end up doing 6'528'600 model runs:

In [5]:



# figure 14: emissions
slct_vr = list(np.arange(0, max([vrid for vrid, vrvl in dict_vre.items() if not vrvl == 'default' and vrvl <= 0.45]), 2) + 1)
slct_st = [0, 20]
mask_emissions = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0, 1]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
#                ml.df_def_loop.swco_id.isin([0]) & --> all
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )

# figure 13: consecutive replacement
slct_vr = list(np.arange(0, max([vrid for vrid, vrvl in dict_vre.items() if not vrvl == 'default' and vrvl <= 0.425]), 1) + 1)
slct_st = [0, 20]#list(np.arange(0, ml.df_def_loop.swvr_id.max() + 5, 10))
mask_consec = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0]) &
                ml.df_def_loop.swpt_id.isin([0, 1, 2]) &
                ml.df_def_loop.swyr_id.isin([0]) &
                ml.df_def_loop.swco_id.isin([0, 1]) &
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )

# figure 12: qualitatively diverging storage impact
slct_vr = [{round(val * 10000) / 10000 if not val == 'default' else val: key
            for key, val in dict_vre.items()}[vr] for vr in [0.5, 0.7, 0.4]]
mask_years = (ml.df_def_loop.swvr_id.isin(slct_vr) &
#                ml.df_def_loop.swst_id.isin(slct_st) & --> all
                ml.df_def_loop.swtc_id.isin([0, 1]) &
                ml.df_def_loop.swpt_id.isin([0]) &
#                ml.df_def_loop.swyr_id.isin([0]) & --> all
                ml.df_def_loop.swco_id.isin([0]) &
                ml.df_def_loop.swtr_id.isin([0]) &
                ml.df_def_loop.swrc_id.isin([0]) 
                )



# figure 11: ramping costs
slct_vr = [{round(val * 10000) / 10000 if not val == 'default' else val: key
            for key, val in dict_vre.items()}[vr] for vr in [0.3, 0.5, 0.7]]
slct_st = [0, 20]#list(np.arange(0, ml.df_def_loop.swvr_id.max() + 10, 10))
mask_ramping = (ml.df_def_loop.swvr_id.isin(slct_vr) &
                ml.df_def_loop.swst_id.isin(slct_st) &
                ml.df_def_loop.swtc_id.isin([0]) &
                ml.df_def_loop.swpt_id.isin([0]) &
                ml.df_def_loop.swyr_id.isin([0]) &
                ml.df_def_loop.swco_id.isin([0]) &
                ml.df_def_loop.swtr_id.isin([0]) 
#               ml.df_def_loop.swrc_id.isin([0]) --> all
               )


# figures 9+10: various nuclear power indicators france 
slct_vr = [0] + list(np.arange(0, ml.df_def_loop.swvr_id.max(), 4) + 1)
slct_st = list(np.arange(0, ml.df_def_loop.swvr_id.max() + 10, 10))
mask_base = (ml.df_def_loop.swvr_id.isin(slct_vr) &
               ml.df_def_loop.swst_id.isin(slct_st) &
               ml.df_def_loop.swtc_id.isin([0, 1]) &
               ml.df_def_loop.swpt_id.isin([0]) &
               ml.df_def_loop.swyr_id.isin([0]) &
               ml.df_def_loop.swco_id.isin([0]) &
               ml.df_def_loop.swtr_id.isin([0]) &
               ml.df_def_loop.swrc_id.isin([0]))


mask_total = mask_base | mask_ramping | mask_years | mask_consec | mask_emissions

ml.df_def_loop = pd.concat([ml.df_def_loop.loc[mask_total]])
ml.df_def_loop

,run_id,swvr_id,swst_id,swtc_id,swpt_id,swyr_id,swco_id,swtr_id,swrc_id,swvr,...,swtr,swrc,swvr_vl,swst_vl,swtc_vl,swpt_vl,swyr_vl,swco_vl,swtr_vl,swrc_vl
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Model loop

The ModelLoopModifier instance makes changes to ModelBase object as specified by the model run and its index combination.

In [8]:
from importlib import reload

reload(model_loop_modifier)

# init ModelLoopModifier
mlm = model_loop_modifier.ModelLoopModifier(ml)

# starting row of loop
irow_0 = ml.io.resume_loop if ml.io.resume_loop else 0

irow = irow = 0

ml.m._limit_prof_to_cap()
#ml.perform_model_run(zero_run=True)
# %
for irow in list(range(irow_0, len(ml.df_def_loop))):
    run_id = irow

    print('select_run')
    ml.select_run(run_id)

    ###
    print('set_trm_cap_onoff')
    mlm.set_trm_cap_onoff()
    
    ####
    print('set_co2_price')
    mlm.set_co2_price()

    ####
    print('set_winsol_year')
    mlm.set_winsol_year()

    ####
    print('select_vre_pp_types')
    slct_pp, slct_pt = mlm.select_vre_pp_types()

    ####
    print('scale_vre')
    mlm.scale_vre(slct_pt, dict_vre)

    ####
    print('set_ramping_cost')
    mlm.set_ramping_cost(dict_rc)

    ####
    print('select_storage_tech')
    mlm.select_storage_tech()

    ####
    print('set_storage_cap')
    mlm.set_storage_cap(dict_st)

    ############### RUN MODEL ###############
    print('fill_peaker_plants')
    ml.m.fill_peaker_plants(demand_factor=2)

    print('_limit_prof_to_cap')
    ml.m._limit_prof_to_cap()

    print('perform_model_run')
    ml.perform_model_run()

    for fn in [fn for fn in os.listdir('.') if 'ephemeral' in fn or 'def_loop' in fn]:
        os.remove(fn)





************************************************************
ModelBase: Limiting chp profiles to cap_pwr_leg... ok, nothing changed.
************************************************************
select_run
set_trm_cap_onoff
set_co2_price
set_winsol_year
select_vre_pp_types
scale_vre
set_ramping_cost
select_storage_tech
set_storage_cap
fill_peaker_plants
_limit_prof_to_cap
************************************************************
ModelBase: Limiting chp profiles to cap_pwr_leg... ok, nothing changed.
************************************************************
perform_model_run
************************************************************
run_id = 0 of 1407
swvr_vl = default
swst_vl = 0.00%
swtc_vl = CAS_STO
swpt_vl = WIN_ONS|WIN_OFF|SOL_PHO
swyr_vl = 2015
swco_vl = 40EUR/tCO2
swtr_vl = on
swrc_vl = x1.0
************************************************************
Using warmstartfile:  None
Using solutionfile:  /tmp/manual_soln_file_replace_all_test_0.cplex.sol

          create_tempfi

/mnt/data/Dropbox/SHARED_REPOS/grimsel/core/io.py:633: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _df['run_id'] = self.run_id


Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.082 sec
Writing to db:  out_replace_all_test.var_yr_erg_fl_yr... done in 0.057 sec
Writing to db:  out_replace_all_test.var_yr_pwr_ramp_yr... done in 0.030 sec
Writing to db:  out_replace_all_test.var_yr_vc_fl_pp_yr... done in 0.045 sec
Writing to db:  out_replace_all_test.var_yr_vc_ramp_yr... done in 0.033 sec
Writing to db:  out_replace_all_test.var_yr_vc_co2_pp_yr... done in 0.025 sec
Writing to db:  out_replace_all_test.var_yr_vc_om_pp_yr... done in 0.059 sec
Writing to db:  out_replace_all_test.var_yr_fc_om_pp_yr... done in 0.060 sec
Component fc_cp_pp_yr does not exist... skipping.
Writing to db:  out_replace_all_test.var_yr_fc_dc_pp_yr... done in 0.061 sec
Writing to db:  out_replace_all_test.var_yr_vc_dmnd_flex_yr... done in 0.019 sec
Writing to db:  out_replace_all_test.var_yr_cap_pwr_rem... done in 0.035 sec
Writing to db:  out_replace_all_test.var_yr_cap_pwr_tot... done in 0.061 sec
Writing to db:  out_replace_a

Writing to db:  out_replace_all_test.par_erg_inp... done in 0.072 sec
Writing to db:  out_replace_all_test.par_erg_chp... done in 0.086 sec
Writing to db:  out_replace_all_test.par_capchnge_max... done in 0.014 sec
Writing to db:  out_replace_all_test.par_price_co2... done in 0.037 sec
Writing to db:  out_replace_all_test.par_vc_fl... done in 0.629 sec
Writing to db:  out_replace_all_test.par_cap_avlb... done in 0.190 sec
Writing to db:  out_replace_all_test.var_tr_erg_trm_rv_yr... done in 0.026 sec
Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.021 sec
Writing to db:  out_replace_all_test.var_tr_erg_trm_sd_yr... done in 0.033 sec
Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.021 sec
select_run
set_trm_cap_onoff
set_co2_price
set_winsol_year
select_vre_pp_types
scale_vre
set_ramping_cost
select_storage_tech
set_storage_cap
fill_peaker_plants
_limit_prof_to_cap
************************************************************
ModelBase: Limiting chp profile

Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.023 sec
Writing to db:  out_replace_all_test.var_mt_erg_mt... done in 0.223 sec
Component share_ws_set does not exist... skipping.
Component co2_int does not exist... skipping.
Writing to db:  out_replace_all_test.par_cap_pwr_leg... done in 0.046 sec
Writing to db:  out_replace_all_test.par_cap_trm_leg... done in 0.122 sec
Writing to db:  out_replace_all_test.par_cf_max... done in 0.030 sec
Writing to db:  out_replace_all_test.par_grid_losses... done in 0.019 sec
Component erg_max does not exist... skipping.
Component hyd_pwr_in_mt_max does not exist... skipping.
Component hyd_pwr_out_mt_min does not exist... skipping.
Writing to db:  out_replace_all_test.par_vc_dmnd_flex... done in 0.018 sec
Writing to db:  out_replace_all_test.par_factor_vc_fl_lin_0... done in 0.022 sec
Writing to db:  out_replace_all_test.par_factor_vc_fl_lin_1... done in 0.022 sec
Writing to db:  out_replace_all_test.par_factor_vc_co2_lin_0... done in 

...


          create_tempfile is monkey patched
          
Solver script file=ephemeralieda.cplex.script
Solver warm-start file=/tmp/manual_soln_file_replace_all_test_1.cplex.sol
Solver log file: '/tmp/manual_log_file_replace_all_test.cplex.sol'
Solver solution file: '/tmp/manual_soln_file_replace_all_test_0.cplex.sol'
Solver problem files: ('ephemeralgbhc.pyomo.lp',)
Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.056 sec
Writing to db:  out_replace_all_test.var_yr_erg_fl_yr... done in 0.048 sec
Writing to db:  out_replace_all_test.var_yr_pwr_ramp_yr... done in 0.035 sec
Writing to db:  out_replace_all_test.var_yr_vc_fl_pp_yr... done in 0.036 sec
Writing to db:  out_replace_all_test.var_yr_vc_ramp_yr... done in 0.035 sec
Writing to db:  out_replace_all_test.var_yr_vc_co2_pp_yr... done in 0.031 sec
Writing to db:  out_replace_all_test.var_yr_vc_om_pp_yr... done in 0.047 sec
Writing to db:  out_replace_all_test.var_yr_fc_om_pp_yr... done in 0.044 sec
Component fc_cp_pp_yr d

Writing to db:  out_replace_all_test.par_vc_ramp... done in 0.034 sec
Writing to db:  out_replace_all_test.par_st_lss_hr... done in 0.020 sec
Writing to db:  out_replace_all_test.par_st_lss_rt... done in 0.018 sec
Component hyd_erg_min does not exist... skipping.
Component week_ror_output does not exist... skipping.
Writing to db:  out_replace_all_test.par_dmnd... done in 0.139 sec
Writing to db:  out_replace_all_test.var_sy_pwr... done in 0.161 sec
Writing to db:  out_replace_all_test.par_erg_inp... done in 0.077 sec
Writing to db:  out_replace_all_test.par_erg_chp... done in 0.079 sec
Writing to db:  out_replace_all_test.par_capchnge_max... done in 0.015 sec
Writing to db:  out_replace_all_test.par_price_co2... done in 0.047 sec
Writing to db:  out_replace_all_test.par_vc_fl... done in 0.643 sec
Writing to db:  out_replace_all_test.par_cap_avlb... done in 0.216 sec
Writing to db:  out_replace_all_test.var_tr_erg_trm_rv_yr... done in 0.023 sec
Writing to db:  out_replace_all_test.var_

Writing to db:  out_replace_all_test.var_yr_fc_dc_pp_yr... done in 0.043 sec
Writing to db:  out_replace_all_test.var_yr_vc_dmnd_flex_yr... done in 0.015 sec
Writing to db:  out_replace_all_test.var_yr_cap_pwr_rem... done in 0.020 sec
Writing to db:  out_replace_all_test.var_yr_cap_pwr_tot... done in 0.042 sec
Writing to db:  out_replace_all_test.var_yr_cap_erg_tot... done in 0.021 sec
Component cap_pwr_new does not exist... skipping.
Writing to db:  out_replace_all_test.var_yr_erg_yr... done in 0.027 sec
Writing to db:  out_replace_all_test.var_mt_erg_mt... done in 0.262 sec
Component share_ws_set does not exist... skipping.
Component co2_int does not exist... skipping.
Writing to db:  out_replace_all_test.par_cap_pwr_leg... done in 0.051 sec
Writing to db:  out_replace_all_test.par_cap_trm_leg... done in 0.101 sec
Writing to db:  out_replace_all_test.par_cf_max... done in 0.024 sec
Writing to db:  out_replace_all_test.par_grid_losses... done in 0.015 sec
Component erg_max does not ex

select_run
set_trm_cap_onoff
set_co2_price
set_winsol_year
select_vre_pp_types
scale_vre
set_ramping_cost
select_storage_tech
set_storage_cap
fill_peaker_plants
_limit_prof_to_cap
************************************************************
ModelBase: Limiting chp profiles to cap_pwr_leg... ok, nothing changed.
************************************************************
perform_model_run
************************************************************
run_id = 15 of 1407
swvr_vl = 35.00%
swst_vl = 0.00%
swtc_vl = CAS_STO
swpt_vl = WIN_ONS|WIN_OFF|SOL_PHO
swyr_vl = 2015
swco_vl = 40EUR/tCO2
swtr_vl = on
swrc_vl = x1.0
************************************************************
Using warmstartfile:  /tmp/manual_soln_file_replace_all_test_0.cplex.sol
Using solutionfile:  /tmp/manual_soln_file_replace_all_test_1.cplex.sol

          create_tempfile is monkey patched
          

          create_tempfile is monkey patched
          
Solver script file=ephemeraleaia.cplex.script
Solver warm-st

KeyboardInterrupt: 

## Basic analysis tables
Prints the executed SQL query.

In [14]:
sqa = sql_analysis.SqlAnalysis(sc_out=sc_out, db=db)
print(sqa.build_tables_plant_run_quick())
print(sqa.build_table_plant_run_tot_balance(from_quick=True))

Calling generate_view_time_series_subset
Join columns pp_broad_cat, pt from out_replace_all_test.def_pp_type to out_replace_all_test.analysis_plant_run_quick 0.058037519454956055 seconds
Join columns swvr_vl, swst_vl, swtc_vl, swpt_vl, swyr_vl, swco_vl, swtr_vl, swrc_vl from out_replace_all_test.def_loop to out_replace_all_test.analysis_plant_run_quick 0.04441046714782715 seconds
Join columns set_def_winsol, fl_id, pp from out_replace_all_test.def_plant to out_replace_all_test.analysis_plant_run_quick 0.046907901763916016 seconds
Join columns fl from out_replace_all_test.def_fuel to out_replace_all_test.analysis_plant_run_quick 0.04640316963195801 seconds
Join columns nd from out_replace_all_test.def_node to out_replace_all_test.analysis_plant_run_quick 0.055498361587524414 seconds

                    /* ADD DERIVED COLUMNS */
                    DROP VIEW IF EXISTS plant_run_quick_01 CASCADE;
                    CREATE VIEW plant_run_quick_01 AS
                    WITH totdmnd AS (


## Advanced analysis

Analysis related to emissions and hourly filtered output requires writing all hourly data.

In [ ]:
# EMISSIONS LIN

slct_run_id = aql.exec_sql('''
                           SELECT run_id FROM {sc}.def_loop
                           WHERE swvr_vl IN (SELECT GENERATE_SERIES(0, 100, 5) || '.00%')
                           AND swst_vl IN ('0.00%', '20.00%')
                           AND swrc_vl = 'x1.0'
                           AND swyr_vl = '2015'
                           AND swpt_vl = 'WIN_ONS|WIN_OFF|SOL_PHO'
                           AND swtr_vl = 'on'
                           '''.format(sc=sc_out), db=db)
slct_run_id = list(np.array(slct_run_id).flatten())

sqa = sql_analysis.SqlAnalysis(sc_out=sc_out, db=db, slct_run_id=slct_run_id)
sqa.analysis_emissions_lin()

In [ ]:
# FILTDIFF FOR FRENCH NUCLEAR
import grimsel.auxiliary.maps as maps

mps = maps.Maps(sc_out, db)

slct_nd_id = [mps.dict_nd_id[nd] for nd in ['FR0']]
slct_run_id = aql.exec_sql('''
                           SELECT run_id FROM {sc}.def_loop
                           WHERE swvr_vl IN (SELECT GENERATE_SERIES(0, 100, 10) || '.00%')
                           AND swst_vl IN ('0.00%', '20.00%')
                           AND swrc_vl = 'x1.0'
                           AND swyr_vl = '2015'
                           AND swpt_vl = 'WIN_ONS|WIN_OFF|SOL_PHO'
                           AND swtr_vl = 'on'
                           AND swco_vl = '40EUR/tCO2'
                           '''.format(sc=sc_out), db=db)
slct_run_id = list(np.array(slct_run_id).flatten())

reload(sql_analysis)
sqac_filt_chgdch = sql_analysis.SqlAnalysis(sc_out=sc_out, db=db,
                                            slct_pt=['NUC_ELC'],
                                            slct_run_id=slct_run_id,
                                            nd_id=slct_nd_id)
self = sqac_filt_chgdch
sqac_filt_chgdch.analysis_agg_filtdiff()
